In [ ]:
# pip install python-kasa flask requests picamera

In [ ]:
'''
sudo iptables -t nat -A PREROUTING -p tcp --dport 80 -j REDIRECT --to-ports 8080 
sudo iptables -t nat -D PREROUTING -p tcp --dport 80 -j REDIRECT --to-ports 8080
''' 

import json # dhcp-lease-list
sec = {'kitchen_bulb_ip':"192.168.x.x", 
       'bedroom_bulb_ip':"192.168.x.x",
       'bedroom_ac_ip'  :"192.168.x.x", 
       'hub_ip'         :"192.168.x.x"} 

#with open('secret.json','w') as fw: json.dump(sec, fw)

In [ ]:
import sqlite3, datetime, time, queue, threading, sys, os, asyncio, kasa

with open('secret.json') as fr: sec = json.load(fr) 

def initialize_database(): 
    con = sqlite3.connect(f'SHT40.{datetime.datetime.fromtimestamp(time.time()).strftime("%Y-%m-%dT%H:%M:%S")}.db')  
    cur = con.cursor() 
    cur.execute('CREATE TABLE IF NOT EXISTS log  (Temperature REAL, Humidity REAL, Photoresistor INTEGER, timestamp REAL);')
    con.commit() 
    con.close()
    
#initialize_database()

In [ ]:
def database(): 
    con = sqlite3.connect('SHT40.2021-09-08T19:29:27.db')  
    cur = con.cursor() 
    while True: 
        opt, foo = q_in.get() # q.get() = opt_foo
        bar = False           # opt is a placeholder 
        try:
            if opt == 'insert':
                cur.execute('INSERT INTO log VALUES (?, ?, ?, ?);', 
                            (foo['Temperature (C)'], foo['Humidity (% rH)'], foo['Photoresistor'], time.time(),))  
            elif opt == 'sql':
                cur.execute(foo['sql'])  
                bar = cur.fetchall()
            elif opt == 'commit': 
                con.commit() 
                bar = 'Database commited'
                print('Database commited')
            elif opt == 'close': 
                con.commit()
                con.close()
                bar = 'Database commited and closed'
                print('Database commited and closed')
        except Exception as e:
            print('sql_exception', e, opt, foo)
            pass
        q_out.put(bar) 
        q_out.join()
        q_in.task_done()

q_in, q_out = queue.Queue(), queue.Queue() 
threading.Thread(target=database, name='thread-1', daemon=True).start() 

def database_operator(opt_foo): 
    q_in.join()        
    q_in.put(opt_foo)  
    bar = q_out.get()  
    q_out.task_done()   
    return bar

In [ ]:
import dateutil, requests
UTC = dateutil.tz.gettz('UTC') 
class HUB:
    def __init__(self, temp_ceil = 28.5, lumi_ceil = 700):
        self.ac              = kasa.SmartPlug(sec['bedroom_ac_ip'])  
        self.temp_ceil       = temp_ceil
        self.bulb            = kasa.SmartBulb(sec['bedroom_bulb_ip'])  
        self.night_mode      = False
        self.lumi_ceil       = lumi_ceil 
        threading.Thread(target=self.update).start()  
        time.sleep(3) 
            
    def update(self):
        lat, lng = (40.7063311, -73.9971931)
        date = datetime.datetime.now().strftime('%Y-%m-%d')
        while 1:
            print('[Daily Sunrise & Sunset Update]')
            r = requests.get(f'https://api.sunrise-sunset.org/json?lat={lat}&lng={lng}&date={date}&formatted=0')
            if r.json()['status'] != 'OK': raise
            self.sunrise_dt = datetime.datetime.fromisoformat(r.json()['results']['sunrise'])
            self.sunset_dt  = datetime.datetime.fromisoformat(r.json()['results']['sunset']) 
            while 1: 
                if self.sunrise_dt.day != datetime.datetime.now().day: 
                    print('A New Day Has Come')
                    break
                asyncio.run(self.ac.update())   
                asyncio.run(self.bulb.update())
                if datetime.datetime.now().astimezone(UTC) > self.sunrise_dt and \
                   datetime.datetime.now().astimezone(UTC) < self.sunset_dt: # Daylight
                    self.night_mode = False 
                else:                                                        # At night
                    self.night_mode = True 
                time.sleep(60) 
            break

hub = HUB()

In [ ]:
import flask, logging

log = logging.getLogger('werkzeug') 
#log.setLevel(logging.ERROR)
 
index_html_string = '''
    <!DOCTYPE html>
    <html>
      <head>
        <meta charset="utf-8">
        <title> hub </title>
        <style>
          #inp {height: 100px; width: 100px}
          #sub {height: 400px; width: 400px}
        </style>
      </head>
      <body> 
        <form method="POST" action="/" enctype="multipart/form-data"> 
          <table> 
            <tr>
              <td>Max Celsius Degress:</td>
              <td><input type="text" name="temp_ceil" value="{{hub.temp_ceil}}" id="inp"/></td>
            </tr> 
            <tr>
              <td id="inp">Night Mode:</td> 
              <td>
                <input type="radio" name="night_mode" value="1" {{'checked' if hub.night_mode else ''}} id="inp">
                <label>ON</label>
                <input type="radio" name="night_mode" value='0' {{'' if hub.night_mode else 'checked'}} id="inp">
                <label>OFF</label> 
              </td> 
            </tr> 
          </table>  
          <br> <input type="submit" value="Submit">
        </form>  
      </body>
    </html> 
'''

app = flask.Flask(__name__)  

@app.route("/", methods=['GET', 'POST'])
def index(): 
    req = flask.request 
    if req.method == 'GET':
        return flask.render_template_string(index_html_string, hub=hub) 
    elif req.method == 'POST': 
        hub.temp_ceil = float(req.form['temp_ceil']) 
        hub.night_mode = int(req.form['night_mode']) 
        return flask.render_template_string(index_html_string, hub=hub) 
    
@app.route("/remote", methods=['GET'])
def remote(): 
    pin = flask.request.args.get('pin', -1)
    if   pin ==  '0' : 
        d = 'Pin 0'
    elif pin ==  '2' :  
        d = 'Pin 2'
    elif pin ==  '4' :  
        d = 'Pin 4'
    elif pin ==  '5' :  
        d = 'Pin 5'
    elif pin == '12' :  
        d = 'Pin 12'
    elif pin == '13' :  
        if hub.ac.is_on:
            asyncio.run(hub.ac.turn_off()) 
            asyncio.run(hub.ac.update())   
            d = 'A/C Off' 
        else:
            asyncio.run(hub.ac.turn_on()) 
            asyncio.run(hub.ac.update())  
            d = 'A/C On' 
    elif pin == '14' : 
        d = 'Pin 14' 
    elif pin == '15' : # 
        if hub.bulb.is_on:
            asyncio.run(hub.bulb.turn_off()) 
            asyncio.run(hub.bulb.update())   
            hub.night_mode = False
            d = 'LightOff'
        else:
            asyncio.run(hub.bulb.turn_on()) 
            asyncio.run(hub.bulb.update())   
            hub.night_mode = True
            d = 'LightON'
    return flask.jsonify({'display':d})
    
@app.route("/log", methods=['POST'])
def log():
    T = flask.request.json['Temperature (C)']
    P = flask.request.json['Photoresistor']
    
    if T > hub.temp_ceil and hub.ac.is_on == False: 
        asyncio.run(hub.ac.turn_on()) 
        asyncio.run(hub.ac.update())  
    elif T < hub.temp_ceil - 1 and hub.ac.is_on == True:
        asyncio.run(hub.ac.turn_off()) 
        asyncio.run(hub.ac.update())  
        
    if hub.night_mode:
        if P > hub.lumi_ceil and hub.bulb.is_on: 
            asyncio.run(hub.bulb.turn_off()) 
            asyncio.run(hub.bulb.update())
        else:
            asyncio.run(hub.bulb.turn_on()) 
            asyncio.run(hub.bulb.update()) 
        
    database_operator(('insert', flask.request.json)) 
    return '' 

@app.route("/sql", methods=['POST'])
def sql():  
    return {'result':database_operator(('sql', flask.request.json)).__repr__() }

@app.route('/heartbeat', methods=['GET']) 
def heartbeat():
    database_operator(('commit', None))  
    return 'Cheers!'

print("http://localhost:8080/")
app.run(sec['hub_ip'],'8080') 
# database_operator(('sql', {'sql':'select * from log'})) 

In [ ]:
database_operator(('commit', ''))

In [ ]:
#!jupyter nbconvert --to script hub.ipynb